In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [33]:
with open('TrainData/train_data_scaled.pkl', 'rb') as f:
    train_data = pickle.load(f)

train_data.head()

,CustomerID,ReportedFraud,DateOfIncident,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentState,IncidentCity,IncidentAddress,...,InsuredAge,InsuredZipCode,InsuredGender,InsuredEducationLevel,InsuredOccupation,InsuredHobbies,CapitalGains,CapitalLoss,PolicyCombinedLimit,PolicySingleLimit
0,20065,N,2015-01-09,Multi-vehicle Collision,Rear Collision,Total Loss,Other,9,4,1404,...,2.399207,471784,MALE,JD,sales,video-games,-0.834616,-0.636247,-0.151765,-0.290132
1,37589,N,2015-02-20,Single Vehicle Collision,Side Collision,Minor Damage,Ambulance,4,3,1675,...,-0.602204,431937,FEMALE,High School,transport-moving,polo,1.021569,0.893521,-1.081375,-0.983964
2,24312,N,2015-01-14,Multi-vehicle Collision,Side Collision,Major Damage,Ambulance,5,2,1546,...,1.648854,479320,FEMALE,PhD,machine-op-inspct,exercise,-0.834616,0.893521,1.397586,1.444450
3,5493,Y,2015-01-07,Multi-vehicle Collision,Side Collision,Major Damage,Ambulance,7,4,1413,...,-1.727734,607763,FEMALE,College,exec-managerial,exercise,0.880455,-1.116315,-1.081375,-0.983964
4,7704,Y,2015-02-26,Multi-vehicle Collision,Side Collision,Major Damage,Other,4,6,1367,...,-1.477616,441783,MALE,Masters,sales,chess,-0.834616,0.893521,-0.151765,-0.290132


In [34]:
train_data['DateOfIncident'] = train_data['DateOfIncident'].astype('string')
train_data['DateOfIncident'] = pd.to_datetime(train_data['DateOfIncident'])


train_data['DateOfPolicyCoverage'] = train_data['DateOfPolicyCoverage'].astype('string')
train_data['DateOfPolicyCoverage'] = pd.to_datetime(train_data['DateOfPolicyCoverage'])

train_data['TimeDelta'] = train_data['DateOfIncident'] - train_data['DateOfPolicyCoverage']
train_data['TimeDelta'] = train_data['TimeDelta'].dt.days
train_data['TimeDelta'] = train_data['TimeDelta'].astype('int64')
train_data.drop(['DateOfIncident', 'DateOfPolicyCoverage'], axis=1, inplace=True)
train_data['TimeDelta'].head()

0    5900
1    5298
2    5318
3    8393
4    4904
Name: TimeDelta, dtype: int64

In [35]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_data['TimeDelta'] = scaler.fit_transform(train_data['TimeDelta'].values.reshape(-1,1))
train_data['TimeDelta'].head()

0    0.469348
1    0.218107
2    0.226454
3    1.509788
4    0.053673
Name: TimeDelta, dtype: float64

In [36]:
with open('models/timedelta_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [37]:
with open('TestData/test_data_scaled.pkl', 'rb') as f:
    test_data = pickle.load(f)

test_data.head()

,CustomerID,DateOfIncident,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentState,IncidentCity,IncidentAddress,IncidentTime,...,InsuredAge,InsuredZipCode,InsuredGender,InsuredEducationLevel,InsuredOccupation,InsuredHobbies,CapitalGains,CapitalLoss,PolicyCombinedLimit,PolicySingleLimit
0,10008,2015-02-05,Multi-vehicle Collision,Front Collision,Minor Damage,Ambulance,5,2,1354,4,...,-1.477616,471704,FEMALE,High School,adm-clerical,base-jumping,1.206101,-1.148558,1.397586,1.444450
1,10010,2015-01-13,Single Vehicle Collision,Rear Collision,Minor Damage,Fire,9,5,1383,16,...,0.148148,455810,FEMALE,MD,prof-specialty,golf,1.216956,-1.456661,-1.081375,-0.983964
2,10015,2015-01-05,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,5,2,2030,20,...,0.023090,461919,MALE,JD,other-service,movies,0.265345,0.893521,-0.151765,-0.290132
3,10020,2015-01-03,Vehicle Theft,Rear Collision,Trivial Damage,None,7,5,1449,10,...,-0.101969,600904,FEMALE,Masters,exec-managerial,video-games,1.643915,0.893521,1.397586,-0.290132
4,1003,2015-02-22,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,5,2,1916,7,...,-1.227498,430632,FEMALE,PhD,sales,board-games,0.435405,0.893521,-1.081375,-0.983964


In [38]:
test_data.dtypes

CustomerID                  int64
DateOfIncident           category
TypeOfIncident           category
TypeOfCollission         category
SeverityOfIncident       category
AuthoritiesContacted     category
IncidentState            category
IncidentCity             category
IncidentAddress          category
IncidentTime                int64
NumberOfVehicles         category
PropertyDamage           category
BodilyInjuries           category
Witnesses                category
PoliceReport             category
AmountOfTotalClaim        float64
AmountOfInjuryClaim       float64
AmountOfPropertyClaim     float64
AmountOfVehicleDamage     float64
InsurancePolicyNumber       int64
CustomerLoyaltyPeriod     float64
DateOfPolicyCoverage     category
InsurancePolicyState     category
Policy_Deductible         float64
PolicyAnnualPremium       float64
UmbrellaLimit             float64
InsuredRelationship      category
VehicleMake              category
VehicleModel             category
VehicleYOM    

In [39]:
test_data['DateOfIncident'] = test_data['DateOfIncident'].astype('string')
test_data['DateOfIncident'] = pd.to_datetime(test_data['DateOfIncident'])


test_data['DateOfPolicyCoverage'] = test_data['DateOfPolicyCoverage'].astype('string')
test_data['DateOfPolicyCoverage'] = pd.to_datetime(test_data['DateOfPolicyCoverage'])

test_data['TimeDelta'] = test_data['DateOfIncident'] - test_data['DateOfPolicyCoverage']
test_data['TimeDelta'] = test_data['TimeDelta'].dt.days
test_data['TimeDelta'] = test_data['TimeDelta'].astype('int64')
test_data.drop(['DateOfIncident', 'DateOfPolicyCoverage'], axis=1, inplace=True)
test_data['TimeDelta'].head()

0    6115
1    1160
2    1633
3    5228
4    5282
Name: TimeDelta, dtype: int64

In [40]:
test_data['TimeDelta'] = scaler.fit_transform(test_data['TimeDelta'].values.reshape(-1,1))
test_data['TimeDelta'].head()

0    0.547906
1   -1.510277
2   -1.313805
3    0.179469
4    0.201899
Name: TimeDelta, dtype: float64

In [41]:
train_data.dtypes

CustomerID                  int64
ReportedFraud            category
TypeOfIncident           category
TypeOfCollission         category
SeverityOfIncident       category
AuthoritiesContacted     category
IncidentState            category
IncidentCity             category
IncidentAddress          category
IncidentTime                int64
NumberOfVehicles         category
PropertyDamage           category
BodilyInjuries           category
Witnesses                category
PoliceReport             category
AmountOfTotalClaim        float64
AmountOfInjuryClaim       float64
AmountOfPropertyClaim     float64
AmountOfVehicleDamage     float64
InsurancePolicyNumber       int64
CustomerLoyaltyPeriod     float64
InsurancePolicyState     category
Policy_Deductible         float64
PolicyAnnualPremium       float64
UmbrellaLimit             float64
InsuredRelationship      category
VehicleMake              category
VehicleModel             category
VehicleYOM               category
InsuredAge    

In [42]:
datatypes = pd.DataFrame(train_data.dtypes)

cat_cols = datatypes[datatypes[0] == 'category'].index
cat_cols = cat_cols.tolist()
cat_cols.remove('ReportedFraud')
cat_cols

['TypeOfIncident',
 'TypeOfCollission',
 'SeverityOfIncident',
 'AuthoritiesContacted',
 'IncidentState',
 'IncidentCity',
 'IncidentAddress',
 'NumberOfVehicles',
 'PropertyDamage',
 'BodilyInjuries',
 'Witnesses',
 'PoliceReport',
 'InsurancePolicyState',
 'InsuredRelationship',
 'VehicleMake',
 'VehicleModel',
 'VehicleYOM',
 'InsuredZipCode',
 'InsuredGender',
 'InsuredEducationLevel',
 'InsuredOccupation',
 'InsuredHobbies']

In [43]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

test_data_label_encoded = test_data.copy()
train_data_label_encoded = train_data.copy()
for col in cat_cols:
    train_data_label_encoded[col] = le.fit_transform(train_data_label_encoded[col])
    test_data_label_encoded[col] = le.transform(test_data_label_encoded[col])
train_data_label_encoded.head()

,CustomerID,ReportedFraud,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentState,IncidentCity,IncidentAddress,IncidentTime,...,InsuredZipCode,InsuredGender,InsuredEducationLevel,InsuredOccupation,InsuredHobbies,CapitalGains,CapitalLoss,PolicyCombinedLimit,PolicySingleLimit,TimeDelta
0,20065,N,0,1,2,3,6,3,360,19,...,577,1,3,11,18,-0.834616,-0.636247,-0.151765,-0.290132,0.469348
1,37589,N,2,2,1,0,1,2,604,14,...,29,0,2,13,14,1.021569,0.893521,-1.081375,-0.983964,0.218107
2,24312,N,0,2,0,0,2,1,490,21,...,692,0,6,6,8,-0.834616,0.893521,1.397586,1.444450,0.226454
3,5493,Y,0,2,0,0,4,3,369,12,...,809,0,1,3,8,0.880455,-1.116315,-1.081375,-0.983964,1.509788
4,7704,Y,0,2,0,3,1,5,325,3,...,160,1,5,11,5,-0.834616,0.893521,-0.151765,-0.290132,0.053673


In [44]:
train_data_label_encoded['ReportedFraud'] = train_data_label_encoded['ReportedFraud'].map({'Y': 1, 'N': 0})
train_data_label_encoded['ReportedFraud'].head()

0    0
1    0
2    0
3    1
4    1
Name: ReportedFraud, dtype: category
Categories (2, int64): [0, 1]

In [63]:
from sklearn.model_selection import train_test_split

X = train_data_label_encoded.drop(['ReportedFraud', 'CustomerID', 'InsurancePolicyNumber'], axis=1)
y = train_data_label_encoded['ReportedFraud']

X_tr, X_test_dev, y_tr, y_test_dev = train_test_split(X, y, test_size=0.2)

In [52]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

def model_scores(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print('Accuracy: ', accuracy_score(y_test, y_pred))
    print('Precision: ', precision_score(y_test, y_pred))
    print('Recall: ', recall_score(y_test, y_pred))
    print('F1 Score: ', f1_score(y_test, y_pred))
    print('ROC AUC Score: ', roc_auc_score(y_test, y_pred))
    print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
    print('Cross Validation Score: ', cross_val_score(model, X_train, y_train, cv=5, scoring='recall').mean())

In [65]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(max_iter=100000)
model_scores(logreg, X_tr, y_tr, X_test_dev, y_test_dev)

Accuracy:  0.7749653259361997
Precision:  0.6478365384615384
Recall:  0.3490932642487047
F1 Score:  0.4537037037037037
ROC AUC Score:  0.6398638669728371
Confusion Matrix: 
 [[3931  293]
 [1005  539]]
Cross Validation Score:  0.36612828724518076


In [66]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
model_scores(rf, X_tr, y_tr, X_test_dev, y_test_dev)

Accuracy:  0.9228502080443828
Precision:  0.9172361427486713
Recall:  0.7823834196891192
F1 Score:  0.8444599790283118
ROC AUC Score:  0.8782892477233476
Confusion Matrix: 
 [[4115  109]
 [ 336 1208]]
Cross Validation Score:  0.7639802867935375


In [67]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()
model_scores(gb, X_tr, y_tr, X_test_dev, y_test_dev)

Accuracy:  0.8580097087378641
Precision:  0.7536738978306508
Recall:  0.697538860103627
F1 Score:  0.7245206861755803
ROC AUC Score:  0.8071027633851469
Confusion Matrix: 
 [[3872  352]
 [ 467 1077]]
Cross Validation Score:  0.7069393976719838


In [68]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(max_iter=100000)
model_scores(sgd, X_tr, y_tr, X_test_dev, y_test_dev)

Accuracy:  0.7444521497919556
Precision:  0.6458333333333334
Recall:  0.10038860103626943
F1 Score:  0.17376681614349773
ROC AUC Score:  0.5401327474878316
Confusion Matrix: 
 [[4139   85]
 [1389  155]]
Cross Validation Score:  0.4264706637104556


In [57]:
from sklearn.svm import LinearSVC

svc = LinearSVC(max_iter=100000, tol = 0.001)
model_scores(svc, X_tr, y_tr, X_test_dev, y_test_dev)

In [73]:
from sklearn.linear_model import RidgeClassifier

ridge = RidgeClassifier(max_iter=100000, tol = 0.001)
model_scores(ridge, X_tr, y_tr, X_test_dev, y_test_dev)

Accuracy:  0.765256588072122
Precision:  0.6733576642335767
Recall:  0.23898963730569947
F1 Score:  0.35277246653919697
ROC AUC Score:  0.5983063716831527
Confusion Matrix: 
 [[4045  179]
 [1175  369]]
Cross Validation Score:  0.25060266161647266


In [70]:
sgd_log = SGDClassifier(loss='log_loss')
model_scores(sgd_log, X_tr, y_tr, X_test_dev, y_test_dev)

Accuracy:  0.730755894590846
Precision:  0.40816326530612246
Recall:  0.012953367875647668
F1 Score:  0.025109855618330196
ROC AUC Score:  0.5030439187863087
Confusion Matrix: 
 [[4195   29]
 [1524   20]]
Cross Validation Score:  0.4390545769949292


In [72]:
sgd_mod_hinge = SGDClassifier(loss='modified_huber')
model_scores(sgd_mod_hinge, X_tr, y_tr, X_test_dev, y_test_dev)

Accuracy:  0.7406380027739251
Precision:  0.5216216216216216
Recall:  0.375
F1 Score:  0.43632253202712884
ROC AUC Score:  0.6246448863636364
Confusion Matrix: 
 [[3693  531]
 [ 965  579]]
Cross Validation Score:  0.4944837921619348


In [75]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(base_estimator=LogisticRegression(max_iter=100000), n_estimators=100)
model_scores(ada, X_tr, y_tr, X_test_dev, y_test_dev)

Accuracy:  0.7647364771151179
Precision:  0.6515397082658023
Recall:  0.2603626943005181
F1 Score:  0.3720499768625636
ROC AUC Score:  0.6047315365441985
Confusion Matrix: 
 [[4009  215]
 [1142  402]]
Cross Validation Score:  0.28440932232965854


In [76]:
from sklearn.tree import DecisionTreeClassifier
ada_tree = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100)
model_scores(ada_tree, X_tr, y_tr, X_test_dev, y_test_dev)

Accuracy:  0.8299237170596394
Precision:  0.6747361887026692
Recall:  0.7040155440414507
F1 Score:  0.6890649762282092
ROC AUC Score:  0.7899812568692102
Confusion Matrix: 
 [[3700  524]
 [ 457 1087]]
Cross Validation Score:  0.682582732852949


In [77]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(max_iter=100000)
model_scores(mlp, X_tr, y_tr, X_test_dev, y_test_dev)

Accuracy:  0.8077323162274619
Precision:  0.7520278099652375
Recall:  0.42033678756476683
F1 Score:  0.5392604902368093
ROC AUC Score:  0.6848369543884439
Confusion Matrix: 
 [[4010  214]
 [ 895  649]]
Cross Validation Score:  0.366603026010552


Models to use RandomForest, GradientBoosting, ADA. 

In [79]:
with open('TrainData/train_data.pkl', 'wb') as f:
    pickle.dump(train_data, f)

with open('TestData/test_data.pkl', 'wb') as f:
    pickle.dump(test_data, f)